In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import datetime as dt

from sqlalchemy import create_engine
import psycopg2
import config as creds

In [3]:
url = "https://ramk-movie-bucket.s3.us-east-2.amazonaws.com/imdb_main.csv"
imdb_main = pd.read_csv(url)
imdb_main.head()

,imdb_id,title,year,duration,country,language,avg_vote,votes,budget,usa_gross_income,...,g_Mystery,g_News,g_Reality_TV,g_Romance,g_Sci_Fi,g_Sport,g_Thriller,g_War,g_Western,director_list
0,tt0010680,The Sentimental Bloke,1919.0,106,Australia,English,6.3,237,NaN,NaN,...,0,0,0,1,0,0,0,0,0,['Raymond Longford']
1,tt0013579,Ombre ammonitrici,1923.0,90,Germany,German,6.8,842,NaN,NaN,...,0,0,0,0,0,0,0,0,0,['Arthur Robison']
2,tt0017938,La glace à trois faces,1983.0,45,France,"None, French",7.0,759,NaN,NaN,...,0,0,0,1,0,0,0,0,0,['Jean Epstein']
3,tt0018054,Il re dei re,1927.0,160,USA,English,7.2,1890,$ 2500000,NaN,...,0,0,0,0,0,0,0,0,0,['Cecil B. DeMille']
4,tt0018742,Il cameraman,1928.0,76,USA,English,8.1,10101,NaN,NaN,...,0,0,0,0,0,0,0,0,0,"['Edward Sedgwick', ' Buster Keaton']"


In [4]:
help(imdb_main.to_sql)

Help on method to_sql in module pandas.core.generic:

to_sql(name: 'str', con, schema=None, if_exists: 'str' = 'fail', index: 'bool_t' = True, index_label=None, chunksize=None, dtype=None, method=None) -> 'None' method of pandas.core.frame.DataFrame instance
    Write records stored in a DataFrame to a SQL database.
    
    Databases supported by SQLAlchemy [1]_ are supported. Tables can be
    newly created, appended to, or overwritten.
    
    Parameters
    ----------
    name : str
        Name of SQL table.
    con : sqlalchemy.engine.(Engine or Connection) or sqlite3.Connection
        Using SQLAlchemy makes it possible to use any DB supported by that
        library. Legacy support is provided for sqlite3.Connection objects. The user
        is responsible for engine disposal and connection closure for the SQLAlchemy
        connectable See `here                 <https://docs.sqlalchemy.org/en/13/core/connections.html>`_.
    
    schema : str, optional
        Specify the sch

In [6]:
imdb_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                70529 non-null  object 
 1   title                  70529 non-null  object 
 2   year                   70529 non-null  float64
 3   duration               70529 non-null  int64  
 4   country                70465 non-null  object 
 5   language               69913 non-null  object 
 6   avg_vote               70529 non-null  float64
 7   votes                  70529 non-null  int64  
 8   budget                 21107 non-null  object 
 9   usa_gross_income       14632 non-null  object 
 10  worlwide_gross_income  30375 non-null  object 
 11  metascore              12858 non-null  float64
 12  reviews_from_users     63624 non-null  float64
 13  reviews_from_critics   60360 non-null  float64
 14  release_year           70529 non-null  float64
 15  ge

In [10]:
lowercase_columns = [col.lower() for col in imdb_main.columns]
imdb_main.columns = lowercase_columns
imdb_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70529 entries, 0 to 70528
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                70529 non-null  object 
 1   title                  70529 non-null  object 
 2   year                   70529 non-null  float64
 3   duration               70529 non-null  int64  
 4   country                70465 non-null  object 
 5   language               69913 non-null  object 
 6   avg_vote               70529 non-null  float64
 7   votes                  70529 non-null  int64  
 8   budget                 21107 non-null  object 
 9   usa_gross_income       14632 non-null  object 
 10  worlwide_gross_income  30375 non-null  object 
 11  metascore              12858 non-null  float64
 12  reviews_from_users     63624 non-null  float64
 13  reviews_from_critics   60360 non-null  float64
 14  release_year           70529 non-null  float64
 15  ge

In [11]:
from config import *
database_string = f"postgresql://{PGUSER}:{PGPASSWORD}@{PGHOST}/{PGDATABASE}"
engine = create_engine(database_string)
imdb_main.to_sql("imdb_main", engine, if_exists = 'append', index = False, chunksize = 20)

In [12]:
url = "https://ramk-movie-bucket.s3.us-east-2.amazonaws.com/tmdb_main.csv"
tmdb_main = pd.read_csv(url)
tmdb_main.head()

,budget,imdb_id,original_language,popularity,revenue,runtime,release_year,orig_lang_cd,collection,website,...,g_Crime,g_Thriller,g_Horror,g_History,g_Mystery,g_War,g_Foreign,g_Music,g_Documentary,g_Western
0,30000000,tt0114709,en,21.946943,373554033,81.0,1995,en,yes,yes,...,0,0,0,0,0,0,0,0,0,0
1,65000000,tt0113497,en,17.015539,262797249,104.0,1995,en,no,no,...,0,0,0,0,0,0,0,0,0,0
2,0,tt0113228,en,11.712900,0,101.0,1995,en,yes,no,...,0,0,0,0,0,0,0,0,0,0
3,16000000,tt0114885,en,3.859495,81452156,127.0,1995,en,no,no,...,0,0,0,0,0,0,0,0,0,0
4,0,tt0113041,en,8.387519,76578911,106.0,1995,en,yes,no,...,0,0,0,0,0,0,0,0,0,0


tmdb_main = tmdb_main[tmdb_main['imdb_id'].notna()]

In [21]:
lowercase_columns = [col.lower() for col in tmdb_main.columns]
tmdb_main.columns = lowercase_columns
tmdb_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37049 entries, 0 to 37049
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             37049 non-null  int64  
 1   imdb_id            37049 non-null  object 
 2   original_language  37043 non-null  object 
 3   popularity         37049 non-null  float64
 4   revenue            37049 non-null  int64  
 5   runtime            36835 non-null  float64
 6   release_year       37049 non-null  int64  
 7   orig_lang_cd       37049 non-null  object 
 8   collection         37049 non-null  object 
 9   website            37049 non-null  object 
 10  genre_name         34934 non-null  object 
 11  g_animation        37049 non-null  int64  
 12  g_comedy           37049 non-null  int64  
 13  g_family           37049 non-null  int64  
 14  g_adventure        37049 non-null  int64  
 15  g_fantasy          37049 non-null  int64  
 16  g_romance          370

In [22]:
# from config import *
# database_string = f"postgresql://{PGUSER}:{PGPASSWORD}@{PGHOST}/{PGDATABASE}"
# engine = create_engine(database_string)
tmdb_main.to_sql("tmdb_main", engine, if_exists = 'append', index = False, chunksize = 20)

In [23]:
import mysql.connector

ModuleNotFoundError: No module named 'mysql'

In [3]:
# import data from AWS RDS database
#def connect():
    
# Set up a connection to the postgres server.
conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
    
conn = psycopg2.connect(conn_string)
print("Connected!")

# Create a cursor object
cursor = conn.cursor()
    

#return conn, cursor


Connected!


In [4]:
movie_test_df = pd.read_sql_query('select * from IMDb_movies', conn)
movie_test_df.head()


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,None,None,None,None,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,None,None,None,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",None,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,None,None,None,None,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,None,None,None,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,None,None,None,None,31.0,14.0
